In [ ]:
import tensorflow as tf
import pydicom
import numpy as np
import os

def array(img):
    dicom = pydicom.dcmread(img)
    dicom_array = dicom.pixel_array * dicom.RescaleSlope + dicom.RescaleIntercept
    return dicom_array.astype(np.float32)  # データ型をfloat32に変換

def WND(X, W):
    R = 255. * (X - W[1] + 0.5 * W[0]) / W[0]
    R[R < 0] = 0
    R[R > 255] = 255
    return R.astype(np.uint8)  # データ型をuint8に変換

# DICOMファイルのパス
gt_folder = '/Users/ryotaroakagawa/pix2pix3D-CT/dicom/dataset01_test/case0029/CT2/'
# gene_folder = '/Users/ryotaroakagawa/pix2pix3D-CT/result/dataset01_test/case0030/dicom/'
gene_folder = '/Users/ryotaroakagawa/pix2pix3D-CT/dicom/dataset01_test/case0029/CT1/'
# 画像の読み込みと処理
gt_files = [gt_folder + f for f in sorted(os.listdir(gt_folder)) if f.endswith('.DCM')]
gene_files = [gene_folder + f for f in sorted(os.listdir(gene_folder)) if f.endswith('.DCM')]

gt_array = np.stack([array(file) for file in gt_files], axis=-1)  # 3つの次元を持つ画像に変換
gene_array = np.stack([array(file) for file in gene_files], axis=-1)  # 3つの次元を持つ画像に変換

gt_img = WND(gt_array, [350, 50])
gene_img = WND(gene_array, [350, 50])

ssim_multiscale = tf.image.ssim_multiscale(gt_img, gene_img, max_val=255.0)

print("Multiscale SSIM:", ssim_multiscale.numpy())
